In [1]:
# from pathlib import Path
# from llama_index import download_loader
# MarkdownReader = download_loader("MarkdownReader")
# loader = MarkdownReader()

In [3]:
# fp_to_tups = {}
# for dp in doc_paths:
#     fp_to_tups[dp] = loader.parse_tups(filepath=Path(dp))

In [32]:
from glob import glob
doc_paths = glob('/home/watsonchua/work/search_engine/docs/govtext/raw_text/*.txt')

In [42]:
from bs4 import BeautifulSoup
import re
fp_to_text = {}
for dp in doc_paths:
    with open(dp, 'r') as f:
        data = f.read()
    text = ''.join(BeautifulSoup(data).findAll(string=True))
    text = re.sub(r"---\nsidebar_position: \d\n---\n\n#\s", '', text)
    text = re.sub(r"import.*;", '', text)
    # Merge hyphenated words
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)
    # # Fix newlines in the middle of sentences
    # text = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", text.strip())
    # Remove multiple newlines
    text = re.sub(r"\n\s*\n", "\n\n", text)
    fp_to_text[dp] = text

In [43]:
fp_to_text

{'/home/watsonchua/work/search_engine/docs/govtext/raw_text/datasets.txt': 'Datasets\n\nGovText is designed to be dataset-centric (i.e. the same dataset can be used for multiple analyses). Thus, you will be required to upload your dataset before proceeding to create an analysis.\n\nYou will find the dataset management page by clicking on the **Datasets** button in the navigation bar.\n\n### DATASETS OVERVIEW\nOn the Datasets page, you can do the following:\n\n* Upload your datasets\n* Update the metadata of an existing dataset, including dataset name, data classification and sensitivity\n* View an overview of your datasets: total size of your raw datasets stored on GovText, size of individual datasets, and number of predictions done on a particular dataset\n* Search for your datasets by name\n* Delete your datasets\n* Download your datasets\n* Navigate to a particular dataset by clicking on a row displayed in the table\n\n> Overview of your datasets\n> \n![Dataset1.JPG](/img/Dataset1.J

In [44]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from ftfy import fix_text
import os
enc = tiktoken.encoding_for_model("gpt-3.5-turbo-0301")

def text_to_docs(fp_to_text, max_chunk_length=400):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name='cl100k_base',
            chunk_size=max_chunk_length,                                                         
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=0,
        )

    counter = 1
    doc_chunks = []

    for filename, text in fp_to_text.items():
        filename_truncated = filename[:-4].split(os.path.sep)[-1]
        # for tup in text:
        #     tup_text = '\n'.join([t for t in tup if t is not None])
        cc = fix_text(text)
        cc_length = len(enc.encode(cc))
        if cc_length == 0:
            continue
        if cc_length > max_chunk_length:
            splits = text_splitter.split_text(cc)
        else:
            splits = [cc]

        # clause_no = cc.split('\n\n')[0]        

        for split_no, s in enumerate(splits):
            source_key = 'C' + "{:05d}".format(counter)
            split_length = len(enc.encode(s))
            doc = Document(page_content=s, metadata={"filename": filename_truncated, "length": split_length, "clause_no": "", "split_no": str(split_no), "source": source_key})
            # Display: filename.strip() + '--' +  clause_no.strip() + '--' + str(split_no)

            doc_chunks.append(doc)
            counter += 1
    return doc_chunks

In [45]:
doc_chunks = text_to_docs(fp_to_text, max_chunk_length=400)

In [39]:
doc_chunks

[Document(page_content='Datasets\n\nGovText is designed to be dataset-centric (i.e. the same dataset can be used for multiple analyses). Thus, you will be required to upload your dataset before proceeding to create an analysis.\n\nYou will find the dataset management page by clicking on the **Datasets** button in the navigation bar.\n\n### DATASETS OVERVIEW\nOn the Datasets page, you can do the following:\n\n* Upload your datasets\n* Update the metadata of an existing dataset, including dataset name, data classification and sensitivity\n* View an overview of your datasets: total size of your raw datasets stored on GovText, size of individual datasets, and number of predictions done on a particular dataset\n* Search for your datasets by name\n* Delete your datasets\n* Download your datasets\n* Navigate to a particular dataset by clicking on a row displayed in the table\n\n> Overview of your datasets\n> \n![Dataset1.JPG](/img/Dataset1.JPG) \n\n### UPLOAD DATASET\nClicking on the **Upload

In [46]:
from open_ai_embeddings import OpenAIEmbeddings
import toml, openai
secrets = toml.load('.streamlit/secrets.toml')

openai_api_key = secrets['openai_api_key_azure']
openai.api_key = openai_api_key
openai.api_type = "azure"
openai.api_base = "https://govtext-ds-experiment.openai.azure.com/"
openai.api_version = "2022-12-01"
# azure_completion_engine = "text-davinci-003-pretrained"
azure_completion_engine = "gpt-35-turbo-0301-pretrained"
azure_embedding_engine = "text-embedding-ada-002-pretrained"

oai_embedder = OpenAIEmbeddings(query_model_name=azure_embedding_engine, document_model_name=azure_embedding_engine, openai_api_key=openai_api_key)

In [47]:
from langchain.vectorstores.faiss import FAISS
db_clauses = FAISS.from_documents(doc_chunks, oai_embedder)
db_clauses.save_local('./govtext_user_guide_docs')

In [48]:
!aws s3 sync ./govtext_user_guide_docs "s3://govtext-qa-index-db/govtext_user_guide_docs"

upload: govtext_user_guide_docs/index.pkl to s3://govtext-qa-index-db/govtext_user_guide_docs/index.pkl
upload: govtext_user_guide_docs/index.faiss to s3://govtext-qa-index-db/govtext_user_guide_docs/index.faiss
